Generate IIIF `manifest.json` (version 2)
===

In [1]:
import json
from jinja2 import Template

In [2]:
urls=[]
json_file = '../src/main/webapp/data/data-remote.json'
with open(json_file, 'r') as fp:
    for item in json.load(fp)['items']:
        for url in item['item']:
            urls.append(url)

In [3]:
urls

['http://localhost:8182/iiif/2/900c341c1c10fff7:MMNAT01_PM_NNM001001033_001/full/max/0/default.jpg',
 'http://localhost:8182/iiif/2/900c341c1c10fff7:MMNAT01_PM_NNM001001033_002/full/max/0/default.jpg',
 'http://localhost:8182/iiif/2/900c341c1c10fff7:MMNAT01_PM_NNM001001033_003/full/max/0/default.jpg',
 'http://localhost:8182/iiif/2/900c341c1c10fff7:MMNAT01_PM_NNM001001033_004/full/max/0/default.jpg',
 'http://localhost:8182/iiif/2/900c341c1c10fff7:MMNAT01_PM_NNM001001033_005/full/max/0/default.jpg',
 'http://localhost:8182/iiif/2/900c341c1c10fff7:MMNAT01_PM_NNM001001033_006/full/max/0/default.jpg',
 'http://localhost:8182/iiif/2/900c341c1c10fff7:MMNAT01_PM_NNM001001033_007/full/max/0/default.jpg',
 'http://localhost:8182/iiif/2/900c341c1c10fff7:MMNAT01_PM_NNM001001033_008/full/max/0/default.jpg']

In [4]:
template = """
{
  "@context": "http://iiif.io/api/presentation/2/context.json",
  "@id": "https://raw.githubusercontent.com/LINNAE-project/SFB-Annotator/14236422d6aac6d75df7ab6753e7da1be8aa1391/data/manifest.json",
  "@type": "sc:Manifest",
  "label": "Sample Field Book",
  "viewingDirection": "left-to-right",
  "viewingHint": "paged",
  {%- set base = urls[0] %}
  {%- set book_id = ':'.join(base.split(':')[:3]) %}
  {%- set seq_id = book_id + '/sequence/default' %}
  "startCanvas": "{{ book_id }}/canvas/p1",
  "sequences": [
    {
      "@id": "{{ book_id }}/sequence/default",
      "@type": "sc:Sequence",
      "label": "Default",
      "canvases": [
      {%- for url in urls %}
        {
          {%- set page = url.split('/')[-5].split('_')[-1] | int() %}
          {%- set img_id = '/'.join(url.split('/')[:-4]) %}
          {#- some fixes required in URLs #}
          {%- set cvs_id = book_id + '/canvas/p' + page|string() %}
          {%- set thb_id = img_id + '/full/!120,120/0/default.jpg' %}
          {%- set res_id = img_id + '/full/max/0/default.jpg' %}
          {%- set srv_id = img_id %}
          {#- set canvas & thumbnail dimensions #}
          {%- set cvs_dim = dict(w=1024, h=768) %}
          {%- set thb_dim = dict(w=120, h=120) %}
          "@id": "{{ cvs_id }}",
          "@type": "sc:Canvas",
          "label": "p. {{ page }}",
          "width": {{ cvs_dim['w'] }},
          "height": {{ cvs_dim['h'] }},
          "thumbnail": {
            "@id" : "{{ thb_id }}",
            "@type": "dctypes:Image",
            "height": {{ thb_dim['w'] }},
            "width": {{ thb_dim['h'] }}
          },
          "images": [
            {
              "@id": "{{ img_id }}",
              "@type": "oa:Annotation",
              "format": "image/jpeg",
              "motivation": "sc:painting",
              "on": "{{ cvs_id }}",
              "resource": {
                "@id": "{{ res_id }}",
                "@type": "dctypes:Image",
                "service": {
                  "@id": "{{ srv_id }}",
                  "profile": "http://iiif.io/api/image/2/level2.json",
                  "protocol": "http://iiif.io/api/image"
                }
              }
            }
          ]
        }
        {{- "," if not loop.last -}}
      {% endfor %}
      ]
    }
  ]
}
"""

j2_template = Template(template)

In [5]:
print(j2_template.render(urls=urls))


{
  "@context": "http://iiif.io/api/presentation/2/context.json",
  "@id": "https://raw.githubusercontent.com/LINNAE-project/SFB-Annotator/14236422d6aac6d75df7ab6753e7da1be8aa1391/data/manifest.json",
  "@type": "sc:Manifest",
  "label": "Sample Field Book",
  "viewingDirection": "left-to-right",
  "viewingHint": "paged",
  "startCanvas": "http://localhost:8182/iiif/2/900c341c1c10fff7/canvas/p1",
  "sequences": [
    {
      "@id": "http://localhost:8182/iiif/2/900c341c1c10fff7/sequence/default",
      "@type": "sc:Sequence",
      "label": "Default",
      "canvases": [
        {
          "@id": "http://localhost:8182/iiif/2/900c341c1c10fff7/canvas/p1",
          "@type": "sc:Canvas",
          "label": "p. 1",
          "width": 1024,
          "height": 768,
          "thumbnail": {
            "@id" : "http://localhost:8182/iiif/2/900c341c1c10fff7:MMNAT01_PM_NNM001001033_001/full/!120,120/0/default.jpg",
            "@type": "dctypes:Image",
            "height": 120,
           